In [1]:
import os
import sys
from PIL import Image
import numpy as np
import tensorflow as tf
import cv2

# Dict containing train ID to RGB colour mappings
from libs.constants import CITYSCAPES_LABEL_COLORS

# Root Directory of the model we downloaded
MODEL_NAME = 'pspnet'

# Full path to frozen graph for the model.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph_opt.pb'

# We are using a model trained on Cityscapes
# so the output is 19 classes

NUM_CLASSES = 19

# Output dimensions
OUTPUT_SIZE=(1025,2049,3)

In [2]:
# Loading a frozen Tensorflow model into memory
segmentation_graph = tf.Graph()
with segmentation_graph.as_default():
    segmentaion_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
        serialized_graph = fid.read()
        segmentaion_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(segmentaion_graph_def, name='')

In [3]:
def decode_train_ids(image):
    """ Helper function to map labels """
    image = np.squeeze(image, axis=0) # predictions will have the shape [1,1024,2048,1]
    rgb_image = np.zeros(OUTPUT_SIZE)
    for train_id in range(NUM_CLASSES):
        rgb_image[np.where((image==train_id).all(axis=2))] = CITYSCAPES_LABEL_COLORS[train_id]
    return rgb_image

def load_image_into_numpy_array(image):
    """ Helper function to read image """
    image = image.resize((2048, 1024))
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)


def run_inference_for_single_image(image, graph):
    """ Function to run inference on a single input image """
    with graph.as_default():
        with tf.Session() as sess:
            # Grab input and output tensors
            image_tensor = tf.get_default_graph().get_tensor_by_name('inputs:0')
            output_tensor = tf.get_default_graph().get_tensor_by_name('predictions:0')
            # Run inference
            predictions = sess.run(output_tensor,
                                feed_dict={image_tensor: image})
    return predictions


def pil_to_cv(image):
    """ Function to convert a PIL image into a cv2 image"""
    cv2_image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    return cv2_image



In [18]:
image = Image.open("tubingen.jpg")

input_size = image.size
print(image.size)

image_np = load_image_into_numpy_array(image)

cv_image = pil_to_cv(image)




(489, 350)


In [7]:
segmentation_mask = run_inference_for_single_image(image_np, segmentation_graph)
rgb_segmentation_mask = decode_train_ids(segmentation_mask)

In [58]:
def produce_masks(image):
    """ Helper function to produce separate masks from segmentation mask with IDs"""
    masks = []
    image = np.squeeze(image, axis=0)
    for i in range(NUM_CLASSES):
        masks.append(image == i)
    return masks

def array_to_pil(arr):
    """ Helper function to convert numpy arrays into PIL images """
    image = Image.fromarray(arr.astype('uint8'))
    return image

def boolean_to_img(data):
    size = data.shape[::-1]
    databytes = np.packbits(data, axis=1)
    return Image.frombytes(mode='1', size=size, data=databytes)

In [59]:
individual_masks = produce_masks(segmentation_mask)
first = individual_masks[2][:,:,0]

p = boolean_to_img(first)
p.show()

In [19]:
output = Image.fromarray(rgb_segmentation_mask.astype('uint8'))
output = output.resize(input_size)

In [20]:
from PIL.ImageFilter import MedianFilter
output = output.filter(MedianFilter(size=9))

In [21]:
output.show()
image.show()

In [ ]:
background = image.convert("RGBA")
overlay = output.convert("RGBA")

new_image = Image.blend(background, overlay, 0.5)
new_image.show()

In [ ]:
rgb_segmentation_mask = decode_train_ids(segmentation_mask)
